In [1]:
import os
import pymysql
import pandas as pd
import requests
from bs4 import BeautifulSoup

host = os.getenv('localhost')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

conn = pymysql.connect(
    host=host,
    port=int(3306),
    user="root",
    passwd="",
    db="recommender_system",
    charset='utf8mb4')



In [2]:
#PARA BUSCAR UNA PELICULA POR SU CODIGO

def searchMovieIfExist( key ):
    
    sql  = "SELECT * FROM tittle_basics WHERE tconst='%s'"%(key)
   
    df = pd.read_sql_query( sql, conn )
    return df.tail(10)


In [3]:
#searchMovieIfExist("tt0000001")

In [4]:
#OBTIENE TODA LAS LISTAS GURDADAS
def getList(name):
    sql  = "SELECT * FROM lists WHERE name like '%s'"%(name)
   
    df = pd.read_sql_query( sql, conn )
    return df

In [5]:
#getList("nombre prueba")

In [6]:
def insertList(name, link, metadata):
    
    with conn.cursor() as cursor:
        # CREA UNA NUEVA FILA
        sql = "INSERT INTO lists ( name, link, metadata ) VALUES ( %s, %s, %s )"
        cursor.execute( sql,( name, link, metadata ) )
        
        #GUARDAMOS LOS CAMBIOS
        conn.commit()
               
    

In [7]:
#insertList("nombre prueba", "link prueba", "meta prueba")

In [11]:


def getListImdb(url):
    
    countPage = 1
    url+="?page=%d"%countPage
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content, 'lxml')
    nIterations=  soup.find("span",{"class":"list-pagination-total-elements"})
    print(nIterations.text)
         
            
    while r.status_code==200:
        print(r.status_code)
        html_content = r.text
        soup = BeautifulSoup(html_content, 'lxml')

        #OBTENGO TODAS LAS LISTAS
        lists = soup.findAll("div", {"class": "list-preview"})

        resultList = []


        #RECORRO LAS LISTAS Y SACO LA INFORMACIÓN
        for list in lists:
            collectorList = []
            #OBTENGO EL NOMBRE DE LA LISTA
            infoList = list.find( "div" , {"class": "list_name"} )
            nameList = infoList.find( "a" )    
            collectorList.append(nameList.text)


            #OBTENGO EL LINK DE LA LISTA

            linkList = infoList.find( 'a', href = True) 
            collectorList.append(linkList['href'])


            #OBTENGO LA METADATA DE LA LISTA
            metadataList = list.find( "div" , {"class": "list_meta"} )
            collectorList.append(metadataList.text)

            resultList.append(collectorList)
        
        print(url)
        countPage+=1
        url=url.split('?',1)[0]
        url+="?page=%d"%countPage
        r = requests.get(url)
        
        

    return resultList


In [12]:
#URL DE LA LISTA A BUSCAR
url='https://www.imdb.com/lists/tt0074486'

#SACAMOS EL CODIGO DE LA PELICULA
tconst=url.split('/',4)[-1]
tconst=tconst.split('?',1)[0]

#limpiamos parametros get
urlSplit= url.split('?',1)[0]

#OBTENEMOS LAS LISTAS
listas = getListImdb(  url) 

#for lista in listas:
#    insertList(lista[0], lista[1], lista[2])
  

998
200
https://www.imdb.com/lists/tt0074486?page=1
200
https://www.imdb.com/lists/tt0074486?page=2
200
https://www.imdb.com/lists/tt0074486?page=3
200
https://www.imdb.com/lists/tt0074486?page=4
